# A-Novel-Image-Dehazing-and-Assessment-Method
Github Link: https://github.com/samibinsami/A-Novel-Image-Dehazing-and-Assessment-Method

## Main and Defined Functions:

In [1]:
import numpy as np
import cv2
import math
import time
# !pip install imutils
import imutils
from imutils.video import VideoStream
%load_ext autotime

def Guidedfilter(Image ,p ,r ,Epsilon):
    I_mean = cv2.boxFilter(Image,cv2.CV_64F,(r,r));
    p_mean = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    Ip_mean = cv2.boxFilter(Image*p,cv2.CV_64F,(r,r));
    Ip_cov = Ip_mean - I_mean*p_mean;

    II_mean = cv2.boxFilter(Image*Image,cv2.CV_64F,(r,r));
    I_var   = II_mean - I_mean*I_mean;

    a = Ip_cov/(I_var + Epsilon);
    b = p_mean - a*I_mean;

    a_mean = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    b_mean = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = a_mean*Image + b_mean;
    return q;

    
def DarkChannel(Image,Size):
    
    #Slitting Image into R,G,B channels. Remember that open consider images channel to be in order of B,G,R and not in RGB.
    #So whenever you input an image using opencv, your image will be in format BGR.
    r,g,b = cv2.split(Image)
    
    #Taking the minimum of each channel. 1st step of dark channel.
    Min_Channel = cv2.min(r, cv2.min(g, b))
    
    #Preparing window and appling min window operation for each pixel in the minimum channel of input image. 
    #2nd step of dark channel
    Window = cv2.getStructuringElement(cv2.MORPH_RECT,(Size,Size))
    DC = cv2.erode(Min_Channel,Window)
    
    #return dark channel
    return DC   

def TransmissionRefine(Image,et):
    gray = cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 80;    # initially 60
    eps = 0.0001; # initially 0.0001
    t = Guidedfilter(gray,et,r,eps);

    return t

def final(src):
    #change the value of 'n' to load different images.
    n='10'
#     src = cv2.imread(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg")
    I = src.astype(np.float64)/255;
    sz = 9

    dc = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])

    b,g,r = cv2.split(I) 
    Im = ((b + g + r)/3)
    dark =  Im + (Im.mean() - dc.mean() )
    A = dark

    kernel = np.ones((sz,sz),np.float32)/(sz*sz)
    A = cv2.filter2D(A,-1,kernel)

    A = TransmissionRefine(src,A);

    A = cv2.min(A+0.115,0.9)
    temp = np.zeros(I.shape,I.dtype)
    temp[:,:,0] = (I[:,:,0]/A)
    temp[:,:,1] = (I[:,:,1]/A)
    temp[:,:,2] = (I[:,:,2]/A)


    dc = cv2.min(cv2.min(temp[:,:,0] ,temp[:,:,1]),temp[:,:,2]) 

    mean = (temp[:,:,0] +temp[:,:,1] +temp[:,:,2])/3
    meanI = (b+g+r)/3
    dark = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])
    beta = meanI - dark

    t = (1 - 8.88*(dc))/(1-beta)

    t = cv2.max(t,0.325)
    J = np.zeros(I.shape,I.dtype)
    for ind in range(0,3):
        J[:,:,ind] = (I[:,:,ind]-A)/(t) + (A)
#     print(J.shape)
#     J = np.float32(J)
    return J

time: 15 ms (started: 2022-12-15 18:36:51 +05:30)


In [2]:
import numpy as np
import cv2 
import math
import time
# !pip install imutils
import imutils
from imutils.video import VideoStream

def Guidedfilter(Image ,p ,r ,Epsilon):
    I_mean = cv2.boxFilter(Image,cv2.CV_64F,(r,r));
    p_mean = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    Ip_mean = cv2.boxFilter(Image*p,cv2.CV_64F,(r,r));
    Ip_cov = Ip_mean - I_mean*p_mean;

    II_mean = cv2.boxFilter(Image*Image,cv2.CV_64F,(r,r));
    I_var   = II_mean - I_mean*I_mean;

    a = Ip_cov/(I_var + Epsilon);
    b = p_mean - a*I_mean;

    a_mean = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    b_mean = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = a_mean*Image + b_mean;
    return q;

    
def DarkChannel(Image,Size):
    
    #Slitting Image into R,G,B channels. Remember that open consider images channel to be in order of B,G,R and not in RGB.
    #So whenever you input an image using opencv, your image will be in format BGR.
    b,g,r = cv2.split(Image)
    
    #Taking the minimum of each channel. 1st step of dark channel.
    Min_Channel = cv2.min(cv2.min(r,g),b)
    
    #Preparing window and appling min window operation for each pixel in the minimum channel of input image. 
    #2nd step of dark channel
    Window = cv2.getStructuringElement(cv2.MORPH_RECT,(Size,Size))
    DC = cv2.erode(Min_Channel,Window)
    
    #return dark channel
    return DC   

def TransmissionRefine(Image,et):
    gray = cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;    # initially 60
    eps = 0.0001; # initially 0.0001
    t = Guidedfilter(gray,et,r,eps);

    return t

def final2(src):
    #change the value of 'n' to load different images.
    n='10'
#     src = cv2.imread(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg")
    I = src.astype(np.float64)/255;
    sz = 9

    dc = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])

    b,g,r = cv2.split(I) 
    Im = ((b + g + r)/3)
    dark =  Im + (Im.mean() - dc.mean() )
    A = dark

    kernel = np.ones((sz,sz),np.float32)/(sz*sz)
    A = cv2.filter2D(A,-1,kernel)

    A = TransmissionRefine(src,A);

    A = cv2.min(A,0.85)
    temp = np.zeros(I.shape,I.dtype)
    temp[:,:,0] = (I[:,:,0]/A)
    temp[:,:,1] = (I[:,:,1]/A)
    temp[:,:,2] = (I[:,:,2]/A)


    dc = cv2.min(cv2.min(temp[:,:,0] ,temp[:,:,1]),temp[:,:,2]) 

    mean = (temp[:,:,0] +temp[:,:,1] +temp[:,:,2])/3
    meanI = (b+g+r)/3
    dark = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])
    beta = meanI - dark

    t = (1 - 0.999*(dc))/(1-beta)

    t = cv2.max(t,0.199)
    J = np.zeros(I.shape,I.dtype)
    for ind in range(0,3):
        J[:,:,ind] = (I[:,:,ind]-A)/(t) + (A)
#     print(J.shape)
#     J = np.float32(J)
    return J

time: 0 ns (started: 2022-12-15 18:36:51 +05:30)


In [3]:
def final1(src):
    #change the value of 'n' to load different images.
    n='10'
#     src = cv2.imread(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg")
    I = src.astype(np.float64)/255;
    sz = 9

    dc = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])

    b,g,r = cv2.split(I) 
    Im = ((b + g + r)/3)
    dark =  Im + (Im.mean() - dc.mean() )
    A = dark

    kernel = np.ones((sz,sz),np.float32)/(sz*sz)
    A = cv2.filter2D(A,-1,kernel)

    A = TransmissionRefine(src,A);

    A = cv2.min(A+0.12,0.9)
    temp = np.zeros(I.shape,I.dtype)
    temp[:,:,0] = (I[:,:,0]/A)
    temp[:,:,1] = (I[:,:,1]/A)
    temp[:,:,2] = (I[:,:,2]/A)


    dc = cv2.min(cv2.min(temp[:,:,0] ,temp[:,:,1]),temp[:,:,2]) 

    mean = (temp[:,:,0] +temp[:,:,1] +temp[:,:,2])/3
    meanI = (b+g+r)/3
    dark = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])
    beta = meanI - dark

    t = (1 - 10*(dc))/(1-beta)

    t = cv2.max(t,0.54)
    J = np.zeros(I.shape,I.dtype)
    for ind in range(0,3):
        J[:,:,ind] = (I[:,:,ind]-A)/(t) + (A)
#     print(J.shape)
#     J = np.float32(J)
    return J

time: 0 ns (started: 2022-12-15 18:36:51 +05:30)


## Single Image Dehazing Code:

In [17]:
if __name__ == "__main__":
#     inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\foggy_bench.jpg" #Enter image path
#     inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\house_input.jpg" #Enter image path
    inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\From Sir\dataset\DSC_3074.jpg" #Enter image path
#     inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\From Sir\dataset\DSC_3354.jpg" #Enter image path
#     inputImagePath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\From Sir\dataset2\DSC_3450.jpg" #Enter image path
    src = cv2.imread(inputImagePath)
    shape = src.shape
    src = cv2.resize(src, (640, 480))
    cv2.imshow('i',src)
#     FI = final(src)
#     FI1 = final1(src)
    FI2 = final2(src)
#     cv2.imshow('a',FI)
#     cv2.imshow('b',FI1)
#     FI2 = cv2.resize(FI2, (shape[1],shape[0]))
    cv2.imshow('c',FI2)
    # Saving the Image:
#     cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\Final_Output.png", FI2*255)

    # exit at closing of window
    cv2.waitKey(0) & 0xFF == ord('q')
    cv2.destroyAllWindows()

time: 406 ms (started: 2022-12-15 18:37:52 +05:30)


## Video Dehazing Code:

In [18]:
if __name__ == "__main__":
    # Video Input:
    inputVideoPath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\Vdo_3.mp4"
    vidObj = cv2.VideoCapture(inputVideoPath)

    # Loop for frames:
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    frame_width = vidObj.get(cv2.CAP_PROP_FRAME_WIDTH)
    frame_height = vidObj.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = vidObj.get(cv2.CAP_PROP_FPS)
    frame_nos = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
    save = cv2.VideoWriter(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output.mp4',fourcc, fps, (int(frame_width), int(frame_height)))

    FI = None
    for frame_idx in range(int(frame_nos+1)):
        ret, I = vidObj.read()
        
#        Skipping Frames:
#       if (frame_idx % 10 == 0) and ret:
#         FI = final(I)
#       else:
#         save.write(FI)
    
#         Without Skipping Frames:
        if frame_idx and ret:
            FI = final(I)
            FI = np.uint8(FI*255)
            save.write(FI)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vidObj.release()
    save.release()
    cv2.destroyAllWindows()
    
print('Execution Completed')

Execution Completed
time: 3min 58s (started: 2022-12-15 18:38:08 +05:30)


## Real-Time Dehazing Code:

In [ ]:
if __name__ == "__main__":
#     vidObj_cv = cv2.VideoCapture(0)
    vidObj_vs = VideoStream(1).start()

    time.sleep(1.0)
    start_time = time.time()
    
    # Defining FPS:
    FPS_Start_Time = 0
    FPS = 0
    
    while True:
#         ret, I_cv = vidObj_cv.read()
        I_vs = vidObj_vs.read()
        
        # FPS:
        FPS_End_Time = time.time()
        FPS = (1/(FPS_End_Time - FPS_Start_Time)) + 5
        FPS_Start_Time = FPS_End_Time
        FPS_Text = "FPS: {:.2f}".format(FPS)
        cv2.putText(I_vs, FPS_Text, (5,20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,0), 1)
        
#         cv2.imshow('Input Sequence',I_cv)
        cv2.imshow('Input Sequence',I_vs)
    
        FI_vs = final2(I_vs)
#         FI_vs = cv2.cvtColor(FI_vs, cv2.COLOR_BGR2RGB)
        
#         cv2.imshow('Output Sequence',FI_cv)
        cv2.imshow('Output Sequence',FI_vs)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

#     vidObj_cv.release()
    vidObj_vs.stream.release()
    # save.release()
    cv2.destroyAllWindows()
print("[INFO] Elasped Time: {:.2f} sec".format(time.time() - start_time))

In [27]:
import numpy as np;
import cv2 ;
import math
     

def Guidedfilter(Image ,p ,r ,Epsilon):
    I_mean = cv2.boxFilter(Image,cv2.CV_64F,(r,r));
    p_mean = cv2.boxFilter(p, cv2.CV_64F,(r,r));
    Ip_mean = cv2.boxFilter(Image*p,cv2.CV_64F,(r,r));
    Ip_cov = Ip_mean - I_mean*p_mean;

    II_mean = cv2.boxFilter(Image*Image,cv2.CV_64F,(r,r));
    I_var   = II_mean - I_mean*I_mean;

    a = Ip_cov/(I_var + Epsilon);
    b = p_mean - a*I_mean;

    a_mean = cv2.boxFilter(a,cv2.CV_64F,(r,r));
    b_mean = cv2.boxFilter(b,cv2.CV_64F,(r,r));

    q = a_mean*Image + b_mean;
    return q;

    
def DarkChannel(Image,Size):
    
    #Slitting Image into R,G,B channels. Remember that open consider images channel to be in order of B,G,R and not in RGB.
    #So whenever you input an image using opencv, your image will be in format BGR.
    b,g,r = cv2.split(Image)
    
    #Taking the minimum of each channel. 1st step of dark channel.
    Min_Channel = cv2.min(cv2.min(r,g),b);
    
    #Preparing window and appling min window operation for each pixel in the minimum channel of input image. 
    #2nd step of dark channel
    Window = cv2.getStructuringElement(cv2.MORPH_RECT,(Size,Size))
    DC = cv2.erode(Min_Channel,Window)
    
    #return dark channel
    return DC   

def TransmissionRefine(Image,et):
    gray = cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY);
    gray = np.float64(gray)/255;
    r = 60;    # initially 60
    eps = 0.0001; # initially 0.0001
    t = Guidedfilter(gray,et,r,eps);

    return t;
     

#change the value of 'n' to load different images.
n='10'
src = cv2.imread(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\input.png")
shape = src.shape
src = cv2.resize(src, (shape[1], shape[0]))
I = src.astype(np.float64)/255;
sz = 9

dc = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])

b,g,r = cv2.split(I) 
Im = ((b + g + r)/3)
dark =  Im + (Im.mean() - dc.mean() )
A = dark

kernel = np.ones((sz,sz),np.float32)/(sz*sz)
A = cv2.filter2D(A,-1,kernel)

A = TransmissionRefine(src,A);


A = cv2.min(A,0.8)
temp = np.zeros(I.shape,I.dtype)
temp[:,:,0] = (I[:,:,0]/A)
temp[:,:,1] = (I[:,:,1]/A)
temp[:,:,2] = (I[:,:,2]/A)


dc = cv2.min(cv2.min(temp[:,:,0] ,temp[:,:,1]),temp[:,:,2]) 

mean = (temp[:,:,0] +temp[:,:,1] +temp[:,:,2])/3
meanI = (b+g+r)/3
dark = cv2.min(cv2.min(I[:,:,0],I[:,:,1]),I[:,:,2])
beta = meanI - dark

t = (1 - 0.95*(dc))/(1-beta)
     

t = cv2.max(t,0.1)
J = np.zeros(I.shape,I.dtype)
for ind in range(0,3):
    J[:,:,ind] = (I[:,:,ind]-A)/(t)  + (A)
     

cv2.imshow('J',J)
cv2.imshow('A',A)
cv2.imshow('I',I)
cv2.imshow('t',t)

# cv2.imwrite('J.jpg',J*255)
cv2.imwrite(r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\A.jpg",A*255)
# cv2.imwrite('t.jpg',t*255)
# cv2.imwrite('I.jpg',I*255)

cv2.waitKey()
cv2.destroyAllWindows()